# Document Loader

In [1]:
from langchain_community.document_loaders import GitLoader

In [2]:
def load_documents(path, branch):
    loader = GitLoader(
        repo_path=path,
        branch=branch
    )
    data = loader.load()
    return data

In [3]:
data = load_documents('/Users/pandoks/Projects/spotlight', 'setup')

Error reading file /Users/pandoks/Projects/spotlight/.test/chroma.sqlite3: [Errno 2] No such file or directory: '/Users/pandoks/Projects/spotlight/.test/chroma.sqlite3'
Error reading file /Users/pandoks/Projects/spotlight/.test/4b7d4c65-831e-43c5-af7d-fdb220498f48/data_level0.bin: [Errno 2] No such file or directory: '/Users/pandoks/Projects/spotlight/.test/4b7d4c65-831e-43c5-af7d-fdb220498f48/data_level0.bin'
Error reading file /Users/pandoks/Projects/spotlight/.test/4b7d4c65-831e-43c5-af7d-fdb220498f48/header.bin: [Errno 2] No such file or directory: '/Users/pandoks/Projects/spotlight/.test/4b7d4c65-831e-43c5-af7d-fdb220498f48/header.bin'
Error reading file /Users/pandoks/Projects/spotlight/.test/4b7d4c65-831e-43c5-af7d-fdb220498f48/index_metadata.pickle: [Errno 2] No such file or directory: '/Users/pandoks/Projects/spotlight/.test/4b7d4c65-831e-43c5-af7d-fdb220498f48/index_metadata.pickle'
Error reading file /Users/pandoks/Projects/spotlight/.test/4b7d4c65-831e-43c5-af7d-fdb220498f4

In [4]:
print(len(data))
data_document = data[3].dict()
print(data_document)

10
{'page_content': '{\n "cells": [\n  {\n   "cell_type": "markdown",\n   "id": "87d5f168-d778-4df6-9fb9-b7429857f4f1",\n   "metadata": {},\n   "source": [\n    "# Document Loader"\n   ]\n  },\n  {\n   "cell_type": "code",\n   "execution_count": 1,\n   "id": "3f209f37-e612-4e79-8099-d5fe2dd87e12",\n   "metadata": {\n    "collapsed": true,\n    "jupyter": {\n     "outputs_hidden": true\n    }\n   },\n   "outputs": [],\n   "source": [\n    "from langchain_community.document_loaders import GitLoader"\n   ]\n  },\n  {\n   "cell_type": "code",\n   "execution_count": 2,\n   "id": "ef23b2ab-4960-48f4-8337-06ec684d3555",\n   "metadata": {\n    "collapsed": true,\n    "jupyter": {\n     "outputs_hidden": true\n    }\n   },\n   "outputs": [],\n   "source": [\n    "def load_documents(path, branch):\\n",\n    "    loader = GitLoader(\\n",\n    "        repo_path=path,\\n",\n    "        branch=branch\\n",\n    "    )\\n",\n    "    data = loader.load()\\n",\n    "    return data"\n   ]\n  },\n  {\

# Document Splitter

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [6]:
def split_text(documents, language, chunk_size, chunk_overlap):
    splitter = RecursiveCharacterTextSplitter.from_language(language=language, chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs = splitter.split_documents(documents)
    return docs

In [7]:
split = split_text(data, "python", 1000, 100)

In [8]:
print(len(split))
print(split)

374
[Document(page_content='.nvim*', metadata={'source': '.gitignore', 'file_path': '.gitignore', 'file_name': '.gitignore', 'file_type': ''}), Document(page_content='{\n "cells": [\n  {\n   "cell_type": "markdown",\n   "id": "87d5f168-d778-4df6-9fb9-b7429857f4f1",\n   "metadata": {},\n   "source": [\n    "# Document Loader"\n   ]\n  },\n  {\n   "cell_type": "code",\n   "execution_count": 1,\n   "id": "3f209f37-e612-4e79-8099-d5fe2dd87e12",\n   "metadata": {},\n   "outputs": [],\n   "source": [\n    "from langchain_community.document_loaders import GitLoader"\n   ]\n  },\n  {\n   "cell_type": "code",\n   "execution_count": 2,\n   "id": "ef23b2ab-4960-48f4-8337-06ec684d3555",\n   "metadata": {},\n   "outputs": [],\n   "source": [\n    "def load_documents(path, branch):\\n",\n    "    loader = GitLoader(\\n",\n    "        repo_path=path,\\n",\n    "        branch=branch\\n",\n    "    )\\n",\n    "    data = loader.load()\\n",\n    "    return data"\n   ]\n  },\n  {\n   "cell_type": "co

In [9]:
texts = [doc.page_content for doc in split]
metadatas = [doc.metadata for doc in split]

# Embeddings

In [10]:
from langchain_community.embeddings.ollama import OllamaEmbeddings

In [11]:
def embed_documents(texts, model):
    embedder = OllamaEmbeddings(model=model)
    embeddings = embedder.embed_documents(texts)
    return embeddings

In [12]:
# embeddings = embed_documents(texts, "llama3")

# Summary Generator

In [13]:
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import Ollama
from langchain_core.documents import Document

In [14]:
summary_prompt = PromptTemplate(
    input_variables=["document_content", "filepath"],
    template="""Summarize this file so that the contents of the files can be queried with a 
    vector database without confusion later on. This is the document ({filepath}): {document_content}
    """
)

In [15]:
def create_summaries(documents, model):
    new_documents = []
    llm_sequence = summary_prompt | Ollama(model=model)
    for document in documents:
        print(document)
        document_dict = document.dict()
        page_content = document_dict["page_content"]
        metadata = document_dict["metadata"]
        summary = llm_sequence.invoke({"document_content": page_content, "filepath": metadata["file_path"]})
        new_document = Document(page_content=summary, metadata=metadata)
        new_documents.append(new_document)
        new_documents.append(document)
    return new_documents

In [ ]:
summaries = create_summaries(split, "llama3")

page_content='.nvim*' metadata={'source': '.gitignore', 'file_path': '.gitignore', 'file_name': '.gitignore', 'file_type': ''}
page_content='{\n "cells": [\n  {\n   "cell_type": "markdown",\n   "id": "87d5f168-d778-4df6-9fb9-b7429857f4f1",\n   "metadata": {},\n   "source": [\n    "# Document Loader"\n   ]\n  },\n  {\n   "cell_type": "code",\n   "execution_count": 1,\n   "id": "3f209f37-e612-4e79-8099-d5fe2dd87e12",\n   "metadata": {},\n   "outputs": [],\n   "source": [\n    "from langchain_community.document_loaders import GitLoader"\n   ]\n  },\n  {\n   "cell_type": "code",\n   "execution_count": 2,\n   "id": "ef23b2ab-4960-48f4-8337-06ec684d3555",\n   "metadata": {},\n   "outputs": [],\n   "source": [\n    "def load_documents(path, branch):\\n",\n    "    loader = GitLoader(\\n",\n    "        repo_path=path,\\n",\n    "        branch=branch\\n",\n    "    )\\n",\n    "    data = loader.load()\\n",\n    "    return data"\n   ]\n  },\n  {\n   "cell_type": "code",\n   "execution_count"

In [ ]:
print(summaries)

# Vector Store

In [ ]:
from langchain_chroma import Chroma
from langchain_core.documents import Document
from hashlib import sha256

In [ ]:
def setup_database(collection_name, embedding_function, persist_directory):
    database = Chroma(
        collection_name=collection_name,
        embedding_function=embedding_function,
        persist_directory=persist_directory
    )
    return database

In [ ]:
database = setup_database("spotlight", OllamaEmbeddings(model="llama3-chatqa"), "/Users/pandoks/Projects/spotlight/.test")

In [ ]:
def add(db, documents):
    documents_with_hashes = []
    for document in documents:
        document = document.dict()
        page_content = document["page_content"]
        hashed_page_content = sha256(page_content.encode("utf-8")).hexdigest()
        print(hashed_page_content)

        new_metadata = document["metadata"]
        new_metadata["hash"] = hashed_page_content

        query_filter = {
            "$and":
                [
                    {'source': new_metadata["source"]},
                    {'file_path': new_metadata["file_path"]},
                    {'file_name': new_metadata["file_name"]},
                    {'file_type': new_metadata["file_type"]},
                    {'hash': new_metadata["hash"]}
                ]
        }
        existing_documents = db.get(where=query_filter)
        print(existing_documents)
        if not len(existing_documents["ids"]):
            documents_with_hashes.append(Document(page_content, metadata=new_metadata))
        
    if not len(documents_with_hashes):
        return
    return db.add_documents(documents_with_hashes)

In [ ]:
def delete(db, metadatas = None):
    if not metadatas:
        to_be_deleted_documents = db.get()
        to_be_deleted_ids = to_be_deleted_documents["ids"]
        db.delete(ids=to_be_deleted_ids)
        return to_be_deleted_ids

    for metadata in metadatas:
        query_filter_list = []
        for key, value in metadata.items():
            query_filter_list.append({key: value})
        query_filter = None
        if len(query_filter_list) == 1:
            query_filter = query_filter_list[0]
        else:
            query_filter = {
                "$and": query_filter_list
            }
        to_be_deleted_documents = db.get(where=query_filter)
        to_be_deleted_ids = to_be_deleted_documents["ids"]
        if not len(to_be_deleted_ids):
            return
        db.delete(ids=to_be_deleted_ids)
        return to_be_deleted_ids

In [ ]:
def update(db, documents):
    metadatas = []
    for document in documents:
        metadata = document.dict()["metadata"]
        if metadata not in metadatas:
            metadatas.append(metadata)
    print(metadatas)
    delete(db, metadatas)
    return add(db, documents)

In [156]:
added_ids = add(database, split)

c9205effb86dc516823576184de6289b22fe2bcabec9821c8d289f424055dbea
{'ids': ['2cd5ec0b-78a4-484b-b5ed-9317ce11070d'], 'embeddings': None, 'metadatas': [{'file_name': '.gitignore', 'file_path': '.gitignore', 'file_type': '', 'hash': 'c9205effb86dc516823576184de6289b22fe2bcabec9821c8d289f424055dbea', 'source': '.gitignore'}], 'documents': ['.nvim*'], 'uris': None, 'data': None}
a29b5858d0a2da42baeaa6182db28e72d576e5c60b1999aaf0e4d8a6068f42a7
{'ids': ['5c34cd29-96f2-4f30-8aa7-88695b1de8df'], 'embeddings': None, 'metadatas': [{'file_name': 'embeddings.py', 'file_path': 'embeddings.py', 'file_type': '.py', 'hash': 'a29b5858d0a2da42baeaa6182db28e72d576e5c60b1999aaf0e4d8a6068f42a7', 'source': 'embeddings.py'}], 'documents': ['import sys\nimport chromadb\nimport argparse\nimport uuid\nimport json\nfrom chromadb.utils import embedding_functions\n\nchromadb_client = None\nembedding_function = None\ncollection = None'], 'uris': None, 'data': None}
b28bd2ae72559fa2e98c6839ac3d417e20303a39eb4aa6eba756

In [ ]:
added_ids = add(database, summaries)

In [171]:
deleted_ids = delete(database, [{'file_type': '.py'}])

In [70]:
deleted_ids = delete(database)

In [38]:
updated_ids = update(database, split)

[{'source': '.gitignore', 'file_path': '.gitignore', 'file_name': '.gitignore', 'file_type': ''}, {'source': 'embeddings.py', 'file_path': 'embeddings.py', 'file_name': 'embeddings.py', 'file_type': '.py'}, {'source': 'lua/spotlight/init.lua', 'file_path': 'lua/spotlight/init.lua', 'file_name': 'init.lua', 'file_type': '.lua'}, {'source': 'retrieval/embeddings/langchain/ollama.py', 'file_path': 'retrieval/embeddings/langchain/ollama.py', 'file_name': 'ollama.py', 'file_type': '.py'}, {'source': 'retrieval/loaders/langchain/git.py', 'file_path': 'retrieval/loaders/langchain/git.py', 'file_name': 'git.py', 'file_type': '.py'}, {'source': 'retrieval/splitters/langchain/code.py', 'file_path': 'retrieval/splitters/langchain/code.py', 'file_name': 'code.py', 'file_type': '.py'}, {'source': 'retrieval/stores/langchain/chroma.py', 'file_path': 'retrieval/stores/langchain/chroma.py', 'file_name': 'chroma.py', 'file_type': '.py'}]
c9205effb86dc516823576184de6289b22fe2bcabec9821c8d289f424055dbea


In [20]:
print(added_ids)

['c47ed45d-94ba-44f1-aee0-2dc5405c3ee2', '428abb56-e076-479e-9077-48fcd620cbad', '3a872450-26cb-45b9-8aa6-0555cdbc8b3a', '43c10404-4f01-425a-bbd3-c0ebf10cd008', 'd46c894c-37d3-478f-94e2-e3fa0fef0c58', 'ff9fe069-81e2-4ed0-90f2-3ed24494def1', 'eb78cc0e-2d54-4f0f-ab24-4056b11e7c1f', 'f93f67f3-05a4-4929-ae67-c60d37b0c3a6', '9db637c2-3668-41bf-ae00-e62ea2f97d9e', 'f1bd77a4-f5be-4c86-bc39-540feae5cf04', '8cf1a154-5b4b-49a1-a67b-23962f2fd48b', '3600e513-4df5-4080-b545-52f0b33aee5c', '705d0754-4e39-4883-bc60-946e9a7b2166', 'dfe75591-9d13-4b55-a537-2313f7dd4133', '3e107c0a-8e07-438f-bc1d-58c8fd52c31b']


In [71]:
print(deleted_ids)

['00026417-7bcf-46f4-8115-26b9e4dffaa5', '0005a6a0-d4ec-442e-99e5-1dbf40fae3f5', '00ae82c2-e088-462a-a7ab-8fdd9f1e3173', '00bf680e-f6f2-4835-a931-5b6e6d91c732', '00d9f257-968d-4fee-bcd3-284c60283126', '0135b20b-0982-4a94-9b43-aeaec0e451d8', '01995c33-bd0d-4640-b424-293d96e130b4', '01e12afd-cf9e-4b0f-bf22-bc7dc2fccc12', '02449c82-072e-4b8e-9e81-8399e04d0f41', '02c52a96-f0e5-4431-baf5-150ca88ecff8', '02d489f7-e89f-4fe1-8a79-c660ad28d41d', '02dfd888-288d-4295-a81e-97779271f015', '02ee7f0b-0086-4fa1-a32a-ba4a5369ae03', '03091fa8-d492-4479-806f-83ee20b10fee', '031285f6-cd29-469b-a40f-e0f143d990f1', '0371ac4a-d39f-4cc6-b269-607aef93be05', '03c29a88-5c08-460c-8969-296c9a90297e', '04114015-1e60-4e2f-a429-4417231f9cb8', '041d0140-d30e-44e3-8834-0ee2f4f11df6', '04491d21-0601-49d6-a3a2-d1039d0e7d53', '047af82f-ade7-49b4-908f-70d579513f01', '04a40cbe-2605-465a-b818-ba02cd1c9c4c', '04ade1e2-143f-4122-ada8-86a5b422cab2', '04f4d932-cbd0-43a1-b58c-a82caf01ab93', '051baaf9-4d1f-4827-9251-91c8fba50bfa',

In [ ]:
print(len(database.get()["ids"]))

In [ ]:
storage = database.get()
metadatas = storage["metadatas"]
seenfiles = set()
for metadata in metadatas:
    filepath = metadata["file_path"]
    if filepath not in seenfiles:
        print(filepath)
        seenfiles.add(filepath)

# Retrieval

In [ ]:
prompt = '''
ollama
'''

### Vector store-backed retriever

In [ ]:
retriever = database.as_retriever(search_type="mmr")

In [ ]:
documents = retriever.invoke(prompt)

### Native database query

In [59]:
documents = database.similarity_search(prompt)

### Parent Document Retriever

In [96]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

#### Full file retrieval

In [98]:
store = InMemoryStore() # ideally make this persistent
retriever = ParentDocumentRetriever(
    vectorstore=database,
    docstore=store,
    child_splitter=RecursiveCharacterTextSplitter.from_language("python", chunk_size=50, chunk_overlap=10),
)

In [99]:
retriever.add_documents(data, ids=None)

In [100]:
list(store.yield_keys())

['91ed3856-fe7e-4d58-b6f7-5f7631cd980d',
 '4e7fb354-1fa1-4e40-b2a1-6ac578f53634',
 'd3d52e22-4213-4fc8-89e8-7baf80e4c60f',
 'd853e76e-47e8-4af2-87f2-3e12f3097fac',
 'b7d6ba54-321c-4235-ac5c-3fa840739782',
 '764e4bed-546b-482a-a819-c989ff8300e7',
 'c5dae50c-e0c5-4c27-acb7-1cf7b40aab05',
 '778f3599-afb3-4d94-a476-26182c8740f4']

In [103]:
small_documents = database.similarity_search(prompt)
print("Small page content:")
print(small_documents[0].page_content)
documents = retriever.invoke(prompt)

Small page content:
.. " --collection-name spotlight --model "


#### Partial file retrieval (if files are way way too big)

In [105]:
store = InMemoryStore() # ideally make this persistent
retriever = ParentDocumentRetriever(
    vectorstore=database,
    docstore=store,
    child_splitter=RecursiveCharacterTextSplitter.from_language("python", chunk_size=50, chunk_overlap=10),
    parent_splitter=RecursiveCharacterTextSplitter.from_language("python", chunk_size=500, chunk_overlap=10)
)

In [106]:
retriever.add_documents(data, ids=None)

In [107]:
list(store.yield_keys())

['a4ac1558-bcd1-4129-8f6e-ccbef5cdd4b1',
 '6d341b55-ed7f-40a8-90e5-e98573c46fa1',
 'ab98df22-f653-4f22-94f3-b83218d61658',
 '2be0340b-8cc9-4138-b5e9-bfb1506fd1db',
 'd964e80a-cb2f-4b55-9c54-2f119564f430',
 '76818e5e-3031-4669-a66c-dfaadcbbb030',
 '8083d365-585e-4b1e-9a38-a15fb7f9e60a',
 '298031b2-1f78-4cf7-abdf-40876531e541',
 '2cfbdaa6-fde9-4746-b614-062e5ee3a813',
 '2a89b588-84c6-4d20-bd46-7522808a0db9',
 'fa07d4ab-5f28-401e-a5b2-4892e03f739f',
 'f31d37a0-6241-4a46-8835-541a9c8f3d29',
 '18108869-0725-4244-a26e-b644810c9e32',
 '9c889688-1e1c-4074-8b3f-0b70c260b139',
 '89be745c-c0aa-4c12-8f21-05d3fa2430ac',
 '8b8af588-fc76-4a41-b9c4-cc349b4ea94d',
 '595c7773-58a9-4293-b1cb-c9ff767ad551',
 'a61a6464-ce12-4430-8830-cfe023e5c5d4',
 'df962edc-dbc8-4122-b658-6065d153ef10',
 '8bcd41f6-d964-4cbe-a2d6-47b87f3be710',
 '52d56d91-315c-41d7-b18b-2f23356fb3ac',
 '3c005c2f-9f40-45c1-bd9a-a2552b0d0ac3',
 'b2e6960b-e7fb-4735-a5f9-56a3b5d20609',
 '3f6d8ed0-6579-4c01-9006-b60d5a5d09c0',
 'd5e056cd-2d39-

In [112]:
small_documents = database.similarity_search(prompt)
print("Small page content:")
print(small_documents[0].page_content)
documents = retriever.invoke(prompt)
print(documents)

Small page content:
.. " --collection-name spotlight --model "
[Document(page_content='local file = io.open(filePath, "r")\n\t\t\t\tif not file then\n\t\t\t\t\tgoto continue\n\t\t\t\tend\n\t\t\t\tlocal content = file:read("*all")\n\t\t\t\tfile:close()\n\t\t\t\tlocal insertCommand = opts.pythonPath\n\t\t\t\t\t.. " "\n\t\t\t\t\t.. embeddings\n\t\t\t\t\t.. " store --db-location "\n\t\t\t\t\t.. directoryLocation\n\t\t\t\t\t.. "/"\n\t\t\t\t\t.. opts.databaseDirectory\n\t\t\t\t\t.. " --collection-name spotlight --model "\n\t\t\t\t\t.. opts.ollamaModel\n\t\t\t\t\t.. " --file-location "\n\t\t\t\t\t.. filePath\n\t\t\t\tlocal handle = io.popen(insertCommand, "w")', metadata={'source': 'lua/spotlight/init.lua', 'file_path': 'lua/spotlight/init.lua', 'file_name': 'init.lua', 'file_type': '.lua'}), Document(page_content='class CodeSplitTextConfig(TypedDict):\n    text: str\n    language: Language\n    chunk_size: Optional[int]\n    chunk_overlap: Optional[int]\n\n\n# https://python.langchain.com/v0

### Multi Query Retriever

In [76]:
from langchain_community.llms import Ollama
import logging
from langchain_core.prompts import PromptTemplate

In [77]:
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [113]:
def parse_questions(questions):
    lines = questions.strip().split("\n")
    filtered_lines = [line for line in lines if line]
    print(filtered_lines)
    return filtered_lines

# only if you ONLY want the file paths and nothing else
# you won't be able to get the partial contexts unless you feed in the whole thing with the file path
def consolidate_files(documents):
    filepaths = set()
    filtered_documents = []
    for document in documents:
        document_dict = document.dict()
        filepath = document_dict["metadata"]["file_path"]
        if filepath not in filepaths:
            filtered_documents.append(document)
            filepaths.add(filepath)
    return filtered_documents

def run_prompts(prompts):
    documents = []
    for prompt in prompts:
        documents.extend(retriever.invoke(prompt))
    return documents

In [114]:
query_prompt = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate 5
    different versions of the given user question to retrieve relevant coding files
    from a vector database and only those questions. By generating multiple perspectives on the user question,
    your goal is to help the user overcome some of the limitations of the distance-based
    similarity search. You want these questions to help the vector database find the most relevant files
    that relate to the user's question kind of like a search engine.
    Do not include give me anything else but the questions in plain text. 
    Do not include an introduction like "Here are 5 of my generated questions".
    Also don't enumerate or bullet each question, so don't give me "1. question1\n2. question2" or "- question1\n- question2".
    Only Provide these alternative questions separated by newlines so that
    it will find code files that are most relevant to this question: {question}"""
)

In [119]:
llm_sequence = query_prompt | Ollama(model="codellama") | parse_questions | run_prompts
documents = llm_sequence.invoke(prompt)

['ollama', '* What kind of programming language is used for ollama?', '* Can you provide an example of how ollama is used in real-life applications?', '* How does ollama differ from other machine learning algorithms in terms of its approach to data analysis and modeling?', '* How can ollama be utilized to predict or classify new data points based on existing data?', '* What are some potential use cases for ollama in industries such as finance, healthcare, or transportation?']


### Metadata Retriever

In [159]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

In [160]:
metadata_field_info = [
    AttributeInfo(
        name="file_name",
        description="The name of the file. Usually describing what the contents of the files are especially when it comes to coding.",
        type="string"
    ),
    AttributeInfo(
        name="file_path",
        description="The relevant file path of the file. The directories usually tell you something about what the file does too including the name for codings.",
        type="string"
    ),
    AttributeInfo(
        name="hash",
        description="The unique hash of contents that are being stored.",
        type="string"
    ),
    AttributeInfo(
        name="source",
        description="Because some of the contents are split up, the source of the contents it he relative path of the file that the content comes from.",
        type="string"
    )
]
document_content_description = "Contents of a file for code."

In [165]:
retriever = SelfQueryRetriever.from_llm(
    Ollama(model="llama3-chatqa"),
    database,
    document_content_description,
    metadata_field_info
)

ValueError: Self query retriever with Vector Store type <class 'langchain_chroma.vectorstores.Chroma'> not supported.

### Custom composite

### Print Results

In [120]:
for document in documents:
    document = document.dict()
    print(document["metadata"]["file_path"])

retrieval/loaders/langchain/git.py
.gitignore
lua/spotlight/init.lua
embeddings.py
.gitignore
retrieval/splitters/langchain/code.py
lua/spotlight/init.lua
lua/spotlight/init.lua
.gitignore
lua/spotlight/init.lua
retrieval/splitters/langchain/code.py
lua/spotlight/init.lua
lua/spotlight/init.lua
embeddings.py
lua/spotlight/init.lua
retrieval/splitters/langchain/code.py
retrieval/loaders/langchain/git.py
embeddings.py
retrieval/splitters/langchain/code.py
lua/spotlight/init.lua
embeddings.py
embeddings.py
retrieval/splitters/langchain/code.py
lua/spotlight/init.lua
